In [46]:
import pandas as pd
import os
import missingno as msno
from sklearn.impute import KNNImputer
from tqdm import tqdm
import numpy as np


In [4]:
tracker = pd.read_csv('data/train_data/tracker.csv')

In [35]:
# combingin the
directory = 'data/train_data/'

# Initialize an empty dataframe
combined_df = pd.DataFrame()

# Loop through files in the directory

for filename in os.listdir(directory):
    if (filename.startswith('z')):  # Assuming the files are in CSV format
        file_path = os.path.join(directory, filename)
        # Read the file into a dataframe
        df = pd.read_csv(file_path)
        # Add a column with the file name
               
        df = df.rename(columns={'Unnamed: 0': 'timestamp'})
                
        # Concatenate the dataframe to the combined dataframe
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')
               
        imputed_data = df.ffill().bfill()
        
        # Convert the result back to a DataFrame
        imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
        imputed_df.set_index('timestamp', inplace=True)
        imputed_df = imputed_df.resample('1h').mean()
        
        imputed_df['File Name'] = filename
        
        imputed_df.to_csv(f'data/train_data_imputed/{filename}', index=True)               
        
        combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.rename(columns={'Unnamed: 0': 'timestamp'})

In [37]:
df_clean_test = pd.read_csv('data/train_data_imputed/z8jfojffxz.csv')


In [39]:
df_clean_test['date'] = pd.to_datetime(df_clean_test['timestamp']).dt.date

In [70]:
tracker = pd.read_csv('data/train_data/tracker.csv')

In [86]:
date_columns

['date']

In [85]:
# Convert columns with 'Date' in their name to date format
date_columns = [col for col in df_clean_test.columns if 'Date' in col or 'date' in col]
df_clean_test[date_columns] = df_clean_test[date_columns].apply(pd.to_datetime, format='%Y-%m-%d')


In [49]:
tracker_filtered = tracker[tracker['API'].isin(df_clean_test['File Name'].str[:-4])]

In [54]:
tracker_filtered.sort_values(by='Install #', inplace=True)

/tmp/ipykernel_162741/438238233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracker_filtered.sort_values(by='Install #', inplace=True)


In [6]:
tracker = tracker.sort_values(by=['API', 'Install #'])

In [7]:
date_cols = [col for col in tracker.columns if 'Date' in col or 'date' in col]
# Convert date columns to datetime, original format is 3/3/2023 0:00, target format is %Y-%m-%d 
for col in date_cols:
    tracker[col] = pd.to_datetime(tracker[col], format='mixed')

In [96]:
combined_df = pd.read_csv('data/train_data/combined_data.csv')

In [ ]:
msno.matrix(combined_df, figsize=(20, 10), fontsize=12)

In [ ]:
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], format='mixed')
combined_df['day'] = combined_df['timestamp'].dt.day


In [88]:
tracker['Startup Date'].info()

<class 'pandas.core.series.Series'>
Index: 205 entries, 0 to 204
Series name: Startup Date
Non-Null Count  Dtype         
--------------  -----         
53 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.2 KB


In [ ]:
percentage = (nan_counts / len(combined_df)) * 100
#print(percentage)
#print(nan_counts)

In [ ]:
# preparing for downsampling
combined_df.set_index('timestamp', inplace=True)

In [ ]:
# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=2)

# Fit and transform the DataFrame
imputed_data = imputer.fit_transform()

# Convert the result back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=df.columns)



In [ ]:
combined_df.head()

In [ ]:
combined_df.resample('1D').mean().interpolate(method='linear', limit_direction='both', inplace=True)

In [ ]:
combined_df.head()

In [30]:
tracker['startup_date'] = pd.to_datetime(tracker['Startup Date'], format='%m/%d/%Y').dt.date
tracker['failure_date'] = pd.to_datetime(tracker['Corrected Failure Date'], format='%m/%d/%Y').dt.date
tracker['pull_date'] = pd.to_datetime(tracker['Pull date'], format='%m/%d/%Y').dt.date

In [31]:
tracker.to_csv('data/train_data/tracker_date_corrected.csv', index=False)

In [35]:
directory = 'data/train_data_imputed/'

# Initialize an empty dataframe
combined_df_imputed = pd.DataFrame()

# Loop through files in the directory

for filename in os.listdir(directory):
    if (filename.startswith('z')):  # Assuming the files are in CSV format
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        combined_df_imputed = pd.concat([combined_df_imputed, df], ignore_index=True)



In [105]:
combined_df_imputed.to_csv('data/train_data_imputed/combined_data_imputed.csv', index=False)


In [102]:
combined_df_imputed['timestamp'] = pd.to_datetime(combined_df_imputed['timestamp'])
combined_df_imputed['timestamp'] = combined_df_imputed['timestamp'].dt.date

In [103]:
combined_df_imputed.dtypes

timestamp                object
pump_intake_pressure    float64
motor_frequency         float64
motor_temp              float64
pump_intake_temp        float64
x_vibration             float64
y_vibration             float64
motor_amps_phase_b      float64
output_amps_phase_b     float64
casing_pressure         float64
tubing_pressure         float64
gas_rate                float64
oil_rate                float64
water_rate              float64
File Name                object
status                  float64
dtype: object

In [108]:
unique_wells = tracker['API'].unique()

for well in unique_wells:
    
    # preparing comparison dfs    
    #df_temp = combined_df_imputed[combined_df_imputed['File Name'] == well]
    combined_df_imputed['timestamp']  = pd.to_datetime(combined_df_imputed['timestamp'])
    combined_df_imputed['timestamp'] = combined_df_imputed['timestamp'].dt.date
    tracker_temp = tracker[tracker['API'] == well]
    
    for date in tracker_temp['failure_date']:
        # Check if the date is in the DataFrame
        if date in combined_df_imputed[combined_df_imputed['File Name'] == well]['timestamp'].values:
            # Update the status column for that date
            combined_df_imputed.loc[(combined_df_imputed['File Name'] == well) & 
                                    (combined_df_imputed['timestamp'] == date), 'status'] = 'failure'
            print(f'well {well} has a failure on {date}')            
        
    
    
    

/tmp/ipykernel_245354/1032786091.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'failure' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df_imputed.loc[(combined_df_imputed['File Name'] == well) &


well z8j7xj31j7 has a failure on 2023-10-08
well z8j7xjz3z3 has a failure on 2021-05-21
well z8j7xjz3z3 has a failure on 2022-05-06
well z8j7xjz78f has a failure on 2022-01-01
well z8j7xjz78f has a failure on 2023-08-14
well z8jfoj1ef3 has a failure on 2021-12-05
well z8jfoj1ef3 has a failure on 2022-07-08
well z8jfoj1ef3 has a failure on 2023-08-10
well z8jfoj1efe has a failure on 2021-06-15
well z8jfoj1efe has a failure on 2023-09-29
well z8jfoj1o7f has a failure on 2021-07-22
well z8jfoj1o7f has a failure on 2021-09-04
well z8jfoj1o7f has a failure on 2023-02-28
well z8jfoj1z3o has a failure on 2023-03-02
well z8jfoje1jf has a failure on 2022-04-04
well z8jfoje1jf has a failure on 2023-02-14
well z8jfoje1o1 has a failure on 2022-07-13
well z8jfoje1o1 has a failure on 2022-07-01
well z8jfoje1o1 has a failure on 2023-04-01
well z8jfoje87x has a failure on 2022-08-30
well z8jfoje87x has a failure on 2022-12-20
well z8jfojee13 has a failure on 2021-11-10
well z8jfojeefx has a failure on

In [109]:
combined_df_imputed.to_csv('data/train_data_imputed/combined_data_imputed.csv', index=False)

In [82]:
df_temp.loc[:, 'timestamp'] = pd.to_datetime(df_temp['timestamp'])

In [110]:
tracker.columns

Index(['Install #', 'Link', 'API', 'Ins Date', 'Startup Date', 'Pull date',
       'Corrected Failure Date', 'Status', 'PSD (ft)', 'Stage Count',
       'Producer', 'Pump Producer Model', 'Gas Handling', 'GasSep/Int',
       'Motor', 'PMM or IM', 'DHG', 'MLE', 'CABLE', 'Cable Status',
       'Cap string', 'Injection Location', 'Desander', 'Joints Filled',
       'Gas Tools', 'Anode/Tubing', 'Clamps', 'Sand Fallback', 'Casing Size',
       'Casing Weight', 'TBG SIZE', 'Failure Y/N', 'Reason for pulling',
       'Component Failed', 'Sub Category', 'Fail reason', 'Comments',
       'Install Notes', 'T2P', 'Pull Notes', 'Exception', 'Teardown Status',
       'Teardown Schedule', 'Report', 'Time to Report', 'Oil', 'Water', 'Gas',
       'Total Production', 'Pump Intake', 'Free Gas', 'Range',
       'Actual Run Time', 'Percent Uptime', 'Number of Shutdowns',
       'Reason for Pull: General', 'Reason for Pull: Specific',
       'Primary Failed Component', 'Primary Failure Subcomponent',
    

In [83]:
df_temp.dtypes

timestamp                object
pump_intake_pressure    float64
motor_frequency         float64
motor_temp              float64
pump_intake_temp        float64
x_vibration             float64
y_vibration             float64
motor_amps_phase_b      float64
output_amps_phase_b     float64
casing_pressure         float64
tubing_pressure         float64
gas_rate                float64
oil_rate                float64
water_rate              float64
File Name                object
status                  float64
dtype: object